In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
from transformers import pipeline
#
# sentiment 情感
# analysis 分析
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I want to fuck u",
        "I like big dick",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9940324425697327},
 {'label': 'POSITIVE', 'score': 0.9993284940719604}]

# 使用 tokenizer/分词器 进行预处理

In [2]:
from transformers import AutoTokenizer
#
# 如你所见 这只是一个string
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
#
# 一个分词器的类
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
#
# 原始输入
raw_inputs = [
    "I want a busty woman to be my wife",
    "I want to fuck",
]
#
# HuggingFace:暂时不用担心填充和截断;我们稍后会解释这些。
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
print(type(inputs))

{'input_ids': tensor([[  101,  1045,  2215,  1037, 13950,  2100,  2450,  2000,  2022,  2026,
          2564,   102],
        [  101,  1045,  2215,  2000,  6616,   102,     0,     0,     0,     0,
             0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}
<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [16]:
#
# 看起来并不是字典类型
# 尝试一下最底层算不算是字典
inputs["input_ids"]
inputs["input_ids"].shape

torch.Size([2, 12])

# 浏览模型

In [19]:
#
# 没什么 就是模型类
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [29]:
#
# 小插曲 但很重要的知识
def a(**k):
    print(type(k))
a(**inputs)
def b(*k):
    print(type(k))
b(*inputs)

<class 'dict'>
<class 'tuple'>


In [20]:
#
# HuggingFace
# NOTE 高维向量 / A high-dimensional vector

# Transformer 模块的 vector 输出通常很大。它通常有三个维度：

# Batch size：一次处理的序列数（在我们的示例中为 2）。

# Sequence length：序列的数字表示长度（在我们的示例中为 16）。

# Hidden size / 隐藏大小 ：每个模型输入的向量维度。

outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 12, 768])


# Model heads: Making sense out of numbers / 模型头：从数字中理解

In [30]:
#
# Transformer 模型的输出直接发送到模型头进行处理。

In [4]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [5]:
print(outputs.logits.shape)

torch.Size([2, 2])


In [6]:
type(outputs)

transformers.modeling_outputs.SequenceClassifierOutput

In [7]:
#
# HF
# 我们的模型预测第一句话为 [-1.5607， 1.6123]，第二句话为 [ 4.1692， -3.3464]。 
# 这些不是概率，而是 logits，即模型最后一层输出的原始、未规范化的 分数 / scores 
print(outputs.logits)

tensor([[-2.5010,  2.5566],
        [ 3.5841, -2.9330]], grad_fn=<AddmmBackward0>)


In [8]:
import torch
#
# 要转换为概率，它们需要经过 SoftMax 层
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

#
# （所有 🤗 Transformers 模型都输出 logits，
# 因为用于训练的损失函数通常会将最后一个激活函数（如 SoftMax）与实际的损失函数（如交叉熵）融合在一起）

tensor([[0.0063, 0.9937],
        [0.9985, 0.0015]], grad_fn=<SoftmaxBackward0>)
